# Single Particle


Simulate the trajectory of an individual point, first in a perfectly circular flow (a.k.a. solid body rotation). Then add a convergent term to obtain a spiraling trajectory, and a constant vertical velocity for the third dimension. These simple flow configurations can be thought of as idealized models e.g. ocean meso-scale eddies.

For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

![solid body rotation](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/SolidBodyRotation.gif)

# 1 Problem Configuration

Here we set up software, grid, flow fields, initial conditions.

### 1.1 Import Software

In [1]:
using IndividualDisplacements, DataFrames
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/helper_functions.jl"))

Main.##259.isosurface

### 1.2  Gridded Domain

In [2]:
np,nz=16,4 #horizontal and vertical domain size
Γ=simple_periodic_domain(np);

### 1.3 Velocity Fields

Exercise: find `simple_flow_field` within `helper_functions.jl` and modify the
flow field parameters (e.g. intensity and sign of the convergent term).

In [3]:
u,v,w=simple_flow_field(Γ,np,nz);

### 1.4 Velocity Function

`🚄` relies only on parameters (velocity fields, grid, etc)
contained in `𝑃` to compute velocity at the space-time position
of the individual. The solver (here: `solv`) can then integrate
over time the result of `🚄` (see `OrdinaryDiffEq.jl` docs).

In [4]:
🚄 = dxyz_dt

𝑃=(u0=u, u1=u, v0=v, v1=v,w0=0.0*w, w1=1.0*w, 𝑇=[0,19.95*2*pi], ioSize=(np,np,nz))

solv(prob) = solve(prob,Tsit5(),reltol=1e-8)

solv (generic function with 1 method)

### 1.5 Initial Position

Here we set up just one individual in a three-dimensional space,

In [5]:
📌=[np*1/3,np*1/3,nz*1/3]

3-element Array{Float64,1}:
 5.333333333333333
 5.333333333333333
 1.3333333333333333

and the data structure ([DataFrame](http://juliadata.github.io/DataFrames.jl/stable/))
to record properties along the individual's path accordingly. It is the postprocessing
function's responsibility to provide the record. It is thus important that this
intermediary (`postproc`) be consistent with the solver setup (`sol`) and
the expected record format (`🔴`).

In [6]:
🔴 = DataFrame(ID=Int[], x=Float64[], y=Float64[], z=Float64[], t=Float64[])

function postproc(sol,𝑃::NamedTuple;id=missing,𝑇=missing)
    df=postprocess_xy(sol,𝑃,id=id,𝑇=𝑇)
    #add third coordinate
    z=sol[3,:]
    df.z=z[:]
    return df
end

postproc (generic function with 1 method)

## 2 Trajectory Simulations

Now that every thing needed to carry out the computation is in place,
we wrap up the problem configuration in a struct (`Individuals`) which
links to the initial positions, flow fields, etc. all that will be
necessary to compute trajectories over time (`∫!(𝐼,𝑇)`). Simple methods to
visualize the individual trajectory (plot or movie) are provided at the end.

### 2.1 Setup Individuals

In [7]:
#assemble as a NamedTuple:
I=(position=📌,record=🔴,velocity=🚄,
integration=solv,postprocessing=postproc,parameters=𝑃)

#construct Individuals from NamedTuple:
𝐼=Individuals(I)

  📌 details     = (3,) Float64
  🔴 details     = (0, 5) ["ID", "x", "y", "z", "t"]
  🆔 range       = (1, 1)
  🚄 function    = dxyz_dt
  ∫  function    = solv
  🔧 function    = postproc
  𝑃  details     = (:u0, :u1, :v0, :v1, :w0, :w1, :𝑇, :ioSize)


### 2.2 Compute Trajectories

The `∫!` function call below returns the final positions & updates `𝐼.📌` accordingly. It also records properties observed along the trajectory in `𝐼.🔴`

In [8]:
𝑇=(0.0,𝐼.𝑃.𝑇[2])
∫!(𝐼,𝑇)

3-element Array{Float64,1}:
 7.767325124766858
 9.513063674566874
 0.7065856129510711

### 2.3 Visualize Trajectories

- define `myplot` convenience function
- generate animation using `myplot`
- single plot example using `myplot`

Single plot example:

Animation example:

Exercise: make the sinking velocity decrease with time
(hint: it increases as specified above in the original notebook);
change the number of times the particle goes around the origin; etc

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*